#Leitura dos dados

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 39 kB/s 
     |████████████████████████████████| 199 kB 53.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e8c9167292a1f40238efc18e6102d301e6f1de9c1446cb91f121d8c4a831c57f
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Transformacao com Spark").getOrCreate()

spark

In [4]:
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip" && unzip semana-2.zip -d dados/

--2022-12-05 12:54:09--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 3.5.19.141, 52.216.220.217, 52.217.200.201, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|3.5.19.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2763652 (2.6M) [application/zip]
Saving to: ‘semana-2.zip’

semana-2.zip        100%[===================>]   2.63M  2.37MB/s    in 1.1s    

2022-12-05 12:54:11 (2.37 MB/s) - ‘semana-2.zip’ saved [2763652/2763652]

Archive:  semana-2.zip
   creating: dados/dataset_transformado_parquet/
  inflating: dados/dataset_transformado_parquet/_SUCCESS  
  inflating: dados/dataset_transformado_parquet/._SUCCESS.crc  
  inflating: dados/dataset_transformado_parquet/part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet  
  inflating: dados/dataset_transformado_parquet/.part-00

In [5]:
dados = spark.read.parquet('/content/dados/dataset_transformado_parquet')

In [6]:
dados.show()
dados.count()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  null|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|null|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

66562

In [7]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



#Analisando a diferença entre a quantidade da dados não nulos armazenados nas colunas 'area_util' e 'area_total'

In [8]:
from pyspark.sql import functions as f

In [9]:
dados.where('area_util == area_total').count()

55384

In [10]:
lista = ['area_util', 'area_total']
dados.select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in lista]).show()

+---------+----------+
|area_util|area_total|
+---------+----------+
|        0|      9194|
+---------+----------+



In [11]:
dados = dados.drop('area_total')

In [12]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



#Convertendo os tipos das colunas

In [13]:
dados = dados.withColumn('area_util', dados.area_util.cast('double'))
dados = dados.withColumn('iptu', dados.iptu.cast('double'))
dados = dados.withColumn('valor', dados.valor.cast('double'))
dados = dados.withColumn('condominio', dados.condominio.cast('double'))

dados = dados.withColumn('andar', dados.andar.cast('integer'))
dados = dados.withColumn('banheiros', dados.banheiros.cast('integer'))
dados = dados.withColumn('quartos', dados.quartos.cast('integer'))
dados = dados.withColumn('suites', dados.suites.cast('integer'))

In [14]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



#Tratamento da coluna caracteristicas

In [15]:
caracteristicas = dados.select(dados.id,f.explode(dados.caracteristicas))
caracteristicas.groupBy('col').count().show()

+------------------+-----+
|               col|count|
+------------------+-----+
|Condomínio fechado|34984|
|        Playground|31856|
| Portão eletrônico|29324|
|           Piscina|33236|
|Animais permitidos|30619|
|      Portaria 24h|30361|
|          Elevador|43027|
|          Academia|27643|
|   Salão de festas|33585|
|     Churrasqueira|31739|
+------------------+-----+



In [16]:
caracteristicas = caracteristicas.groupBy('id').pivot('col').agg(f.lit(1)).na.fill(0)
caracteristicas.show()

+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|       1|                 1|            1|                 1|       1|      1|         1|           1|                1|              1|
|bfffedfe-99e7-4ae...|       0|                 1|            1|                 1|       1|      0|         1|           0|                1|              1|
|fcb67af3-5601-415...|       1|                 1|            1|                 0|       0|      1|         1|           0|                0|              1|
|afecddff-f4cc-4ab...|       1|               

In [17]:
dados = dados.join(caracteristicas, 'id', how='left').drop('caracteristicas')
dados.count()

66562

In [18]:
dados.show()

+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+------+-----+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|  iptu| tipo|   valor|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+------+-----+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|03a386b6-7ab8-4ef...|    0|     43.0|        1|      2|  null|       

#Tratar dados nulos e NaN's

In [19]:
dados.select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dados.columns]).show(vertical=True)

-RECORD 0-------------------
 id                 | 0     
 andar              | 0     
 area_util          | 0     
 banheiros          | 0     
 quartos            | 0     
 suites             | 5554  
 tipo_anuncio       | 0     
 tipo_unidade       | 0     
 tipo_uso           | 0     
 vaga               | 3017  
 bairro             | 0     
 zona               | 0     
 condominio         | 2371  
 iptu               | 7199  
 tipo               | 0     
 valor              | 0     
 Academia           | 12736 
 Animais permitidos | 12736 
 Churrasqueira      | 12736 
 Condomínio fechado | 12736 
 Elevador           | 12736 
 Piscina            | 12736 
 Playground         | 12736 
 Portaria 24h       | 12736 
 Portão eletrônico  | 12736 
 Salão de festas    | 12736 



In [20]:
dados = dados.fillna(value =0)

In [21]:
dados.select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dados.columns]).show(vertical=True)

-RECORD 0-----------------
 id                 | 0   
 andar              | 0   
 area_util          | 0   
 banheiros          | 0   
 quartos            | 0   
 suites             | 0   
 tipo_anuncio       | 0   
 tipo_unidade       | 0   
 tipo_uso           | 0   
 vaga               | 0   
 bairro             | 0   
 zona               | 0   
 condominio         | 0   
 iptu               | 0   
 tipo               | 0   
 valor              | 0   
 Academia           | 0   
 Animais permitidos | 0   
 Churrasqueira      | 0   
 Condomínio fechado | 0   
 Elevador           | 0   
 Piscina            | 0   
 Playground         | 0   
 Portaria 24h       | 0   
 Portão eletrônico  | 0   
 Salão de festas    | 0   



#Preparação dos dados para machine learning

In [22]:
dados.groupBy('zona').count().show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
|            |   11|
+------------+-----+



In [23]:
dados = dados.where(f.col('zona') != '')

In [24]:
dados.groupBy('zona').count().show()

+------------+-----+
|        zona|count|
+------------+-----+
|  Zona Norte|11897|
|  Zona Oeste|32979|
|Zona Central| 1144|
|    Zona Sul|20531|
+------------+-----+



In [25]:
zona = dados.groupBy('id').pivot('zona').agg(f.lit(1)).na.fill(0)
zona

DataFrame[id: string, Zona Central: int, Zona Norte: int, Zona Oeste: int, Zona Sul: int]

In [26]:
dados = dados.join(zona, 'id', how='left').drop('zona')
dados.show()
dados.count()

+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------------+----------+------+-----+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------+----------+----------+--------+
|                  id|andar|area_util|banheiros|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|condominio|  iptu| tipo|   valor|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------------+----------+------+-----+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------+---------

66551

#Salvando a base de dados em parquet

In [27]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = false)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = false)
 |-- iptu: double (nullable = false)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = false)
 |-- Academia: integer (nullable = true)
 |-- Animais permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- Condomínio fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- Portaria 24h: integer (nullable = true)
 |-- Portão eletrônico: integer (nullable = true

In [28]:
dados = dados.drop('tipo_anuncio', 'tipo_unidade', 'tipo_uso', 'tipo', 'bairro')

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
dados.write.parquet(
    path='/content/drive/MyDrive/Challenge_Data_Science_Alura/dados/dataset_parquet',
    mode='overwrite',
)